In [4]:
import torch
import torch.onnx
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import onnx
from onnx_coreml import convert

In [5]:
from session import *

ImportError: No module named tqdm

In [3]:
num_classes = 8

In [10]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, num_classes),
    nn.Sigmoid()
)

In [11]:
criterion = nn.BCELoss()
optim_fn = optim.Adam
sess = Session(model_ft, criterion, optim_fn, 1e-2)

In [12]:
sess.load('TrainLayer1')

In [13]:
imsize = 224
dummy_input = torch.randn(1, 3, 224, 224, requires_grad=True)

In [14]:
input_names = ["image"]
output_names = ["classification"]

In [15]:
sess.model.train(False)
torch.onnx.export(sess.model, dummy_input, 'roaddamage.proto', verbose=True)


In [ ]:
model = onnx.load('roaddamage.proto')
coreml_model = convert(model, 'classifier', image_input_names=['inputImage'])
coreml_model.save('roaddamage.mlmodel')

args = dict(is_bgr=False, red_bias = -(0.485 * 255.0)  , green_bias = -(0.456 * 255.0)  , blue_bias = -(0.406 * 255.0))

coreml_model = convert(
    model,
    'classifier',
    image_input_names=['image'],
    image_output_names=['classification'],
    class_labels=[i for i in range(8)],
    preprocessing_args=args
)
coreml_model.save('SplitModel.mlmodel')
